# Webscrapping with Selenium to fill the NAs 

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common import keys 
from time import sleep 
from tqdm import tqdm

import numpy as np


In [3]:
import pandas as pd
df = pd.read_csv('vgsales_copy.csv', index_col = 0).copy()

In [4]:
df.isna().sum()

Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

In [5]:
year_na = df[df['Year'].isna()]

In [6]:
# for years
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.google.com')
driver.find_element(By.ID, 'L2AGLb').click()
for i, row in tqdm(year_na.iterrows(), total=len(year_na)):
    bar = driver.find_element(By.NAME, 'q')
    bar.clear()
    sleep(1)
    bar.send_keys(row['Name'], ' ', row['Platform'], ' year')
    sleep(2)
    driver.find_element(By.NAME, 'q').send_keys(keys.Keys.ENTER)
    try:
        year_box = driver.find_element(By.CLASS_NAME, 'kp-hc').text
        year_na.loc[i, 'Year'] = year_box
    except: 
        year_na.loc[i, 'Year'] = 'null'


C:\Users\ttttt\AppData\Local\Temp\ipykernel_14252\4170003095.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
100%|██████████| 271/271 [21:34<00:00,  4.78s/it]


In [7]:
year_na.Year = year_na.Year.apply(lambda row: row[-4:])

C:\Users\ttttt\AppData\Local\Temp\ipykernel_14252\376746970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_na.Year = year_na.Year.apply(lambda row: row[-4:])


In [ ]:
df['Year'].fillna(year_na['Year'],inplace = True)

In [8]:
df.to_csv('vgsales_afterwebsc.csv')